<a href="https://colab.research.google.com/github/jcmachicao/pucp_energy_efficiency_AI/blob/main/02_NeuralNetworks/hvac_data_02_prediction_neuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
# Replace with the actual URL of your CSV file
url = 'https://github.com/jcmachicao/pucp_energy_efficiency_AI/raw/refs/heads/main/00_data/HVAC_dataset.xlsx'
data = pd.read_excel(url)
data.head(2)

,Date,on_off,damper,active_energy,co2_1,amb_humid_1,active_power,pot_gen,high_pressure_1,high_pressure_2,low_pressure_1,low_pressure_2,high_pressure_3,low_pressure_3,outside_temp,outlet_temp,inlet_temp,summer_setpoint_temp,winter_setpoint_temp,amb_temp_2
0,2022-01-06 00:00:00,0.0,0.0,363150.69,975.0,35.0,0.36,4.0,15.09,15.0,15.27,15.1,15.2,15.2,23.4,24.2,22.8,22.5,18.0,23.4
1,2022-01-06 00:05:00,0.0,0.0,NaN,977.0,35.0,0.36,NaN,15.03,14.9,15.21,15.0,15.2,15.1,23.3,24.2,22.7,22.5,18.0,23.3


In [24]:
data.fillna(method="bfill", inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25632 entries, 0 to 25631
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  25632 non-null  object 
 1   on_off                25632 non-null  float64
 2   damper                25632 non-null  float64
 3   active_energy         25630 non-null  float64
 4   co2_1                 25632 non-null  float64
 5   amb_humid_1           25632 non-null  float64
 6   active_power          25632 non-null  float64
 7   pot_gen               25630 non-null  float64
 8   high_pressure_1       25632 non-null  float64
 9   high_pressure_2       25632 non-null  float64
 10  low_pressure_1        25632 non-null  float64
 11  low_pressure_2        25632 non-null  float64
 12  high_pressure_3       25632 non-null  float64
 13  low_pressure_3        25632 non-null  float64
 14  outside_temp          25632 non-null  float64
 15  outlet_temp        

<ipython-input-24-58fa1ef74984>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method="bfill", inplace=True)


In [25]:
data.Date.isna().sum()

0

In [5]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce', infer_datetime_format=True)
data.set_index('Date', inplace=True)

<ipython-input-5-cf9c410c4bfa>:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data['Date'] = pd.to_datetime(data['Date'], errors='coerce', infer_datetime_format=True)


In [6]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [7]:
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
df.index = data.index

In [8]:
df.index.month.unique()

Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int32', name='Date')

In [9]:
df.columns

Index(['on_off', 'damper', 'active_energy', 'co2_1', 'amb_humid_1',
       'active_power', 'pot_gen', 'high_pressure_1', 'high_pressure_2',
       'low_pressure_1', 'low_pressure_2', 'high_pressure_3', 'low_pressure_3',
       'outside_temp', 'outlet_temp', 'inlet_temp', 'summer_setpoint_temp',
       'winter_setpoint_temp', 'amb_temp_2'],
      dtype='object')

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

## Neural Network

In [11]:
features_nn = ["inlet_temp", "outside_temp", "amb_humid_1", 'active_power', 'high_pressure_1', 'low_pressure_1']

In [12]:
output_variable_nn = 'co2_1'

In [13]:
target = output_variable_nn

In [14]:
X = df[features_nn]
y = df[target]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True)

In [16]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((19224, 6), (6408, 6), (19224,), (6408,))

In [17]:
X.isna().sum(), y.isna().sum()

(inlet_temp         0
 outside_temp       0
 amb_humid_1        0
 active_power       0
 high_pressure_1    0
 low_pressure_1     0
 dtype: int64,
 0)

In [18]:
import torch
import torch.nn as nn

# Define the neural network
class SimpleRegressionNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRegressionNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 4)
        self.fc4 = nn.Linear(4, output_size)
        self.relu = nn.LeakyReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [19]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1

# Create the model
model_nn = SimpleRegressionNet(input_size, hidden_size, output_size)

#criterion = nn.MSELoss()
criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model_nn.parameters(), lr=0.001)

In [20]:
inputs = torch.tensor(X_train.values, dtype=torch.float32)
labels = torch.tensor(y_train.values, dtype=torch.float32)

In [21]:
epochs = 200

# Training loop
for epoch in range(epochs):
    # Forward pass
    outputs = model_nn(inputs)
    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:1040: UserWarning: Using a target size (torch.Size([19224])) that is different to the input size (torch.Size([19224, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Epoch [10/200], Loss: 0.1507
Epoch [20/200], Loss: 0.1259
Epoch [30/200], Loss: 0.0897
Epoch [40/200], Loss: 0.0459
Epoch [50/200], Loss: 0.0207
Epoch [60/200], Loss: 0.0233
Epoch [70/200], Loss: 0.0198
Epoch [80/200], Loss: 0.0197
Epoch [90/200], Loss: 0.0192
Epoch [100/200], Loss: 0.0190
Epoch [110/200], Loss: 0.0187
Epoch [120/200], Loss: 0.0186
Epoch [130/200], Loss: 0.0185
Epoch [140/200], Loss: 0.0185
Epoch [150/200], Loss: 0.0185
Epoch [160/200], Loss: 0.0185
Epoch [170/200], Loss: 0.0185
Epoch [180/200], Loss: 0.0185
Epoch [190/200], Loss: 0.0185
Epoch [200/200], Loss: 0.0185


In [26]:
validation_data = torch.tensor(X_test.values, dtype=torch.float32)
validation_labels = torch.tensor(y_test.values, dtype=torch.float32)

In [27]:
import torch.nn.functional as F

with torch.no_grad():
    # Calculate predictions on the validation set
    predictions = model_nn(validation_data).squeeze(1)
    loss = criterion(predictions, validation_labels)
    mse = F.mse_loss(predictions, validation_labels)
    rmse = torch.sqrt(mse)
    mae = F.l1_loss(predictions, validation_labels)

    print(f"Validation Loss: {loss.item():.4f}")
    print(f"MSE: {mse.item():.4f}")
    print(f"RMSE: {rmse.item():.4f}")
    print(f"MAE: {mae.item():.4f}")

Validation Loss: 0.0187
MSE: 0.0375
RMSE: 0.1935
MAE: 0.1638


### References

https://www.kaggle.com/code/kiritowu/aircon-classification-w-temperature-humidity/notebook

In [31]:
# Save the model's state dictionary
path_model = 'drive/My Drive/00 data/'
torch.save(model_nn.state_dict(), path_model + 'model_nn.pth')

In [32]:
# To load the model later:
model_nn = SimpleRegressionNet(input_size, hidden_size, output_size)
model_nn.load_state_dict(torch.load(path_model + 'model_nn.pth'))
model_nn.eval() # Set the model to evaluation mode

<ipython-input-32-b54a1b335f0b>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_nn.load_state_dict(torch.load(path_model + 'model_nn.pth'))


SimpleRegressionNet(
  (fc1): Linear(in_features=6, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=1, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
)